This is a first pass at implementing buffaloscript. Buffalos are interpreted as Minsky Program Machine instructions, which is used to run the program.

In [200]:
# The Program Machine program we are aiming for
src = """buffalo 2
Buffalo 0

JZ buffalo 4
DEC buffalo
INC Buffalo
JZ buffalo 0
"""

In [206]:
program = "Buffalo buffalo buffalo buffalo Buffalo buffalo buffalo. Buffalo buffalo buffalo buffalo. Buffalo buffalo Buffalo buffalo buffalo buffalo buffalo buffalo buffalo buffalo Buffalo buffalo. Buffalo buffalo buffalo buffalo buffalo buffalo buffalo buffalo. Buffalo buffalo buffalo buffalo Buffalo buffalo Buffalo buffalo buffalo buffalo. Buffalo buffalo Buffalo buffalo buffalo buffalo buffalo buffalo."

In [207]:
from buffaloparse import parse, parses

def buf_to_binary(string):
    if string == "Buffalo":
        return 1
    elif string == "buffalo":
        return 0
    else:
        raise TypeError('must be Buffalo or buffalo')

def reverse_binary_to_decimal(bin_list):
    decimal = 0
    for i, val in enumerate(bin_list):
        decimal += int(val)*2**(i)
    return decimal

def reverse_binary_to_decimal_test():
    assert binary_to_decimal(list('0')) == 0
    assert binary_to_decimal(list('1')) == 1
    assert binary_to_decimal(list('01')) == 2
    assert binary_to_decimal(list('110')) == 3
    assert binary_to_decimal(list('001')) == 4
    assert binary_to_decimal(list('101')) == 5
    assert binary_to_decimal(list('101000000')) == 5

binary_to_decimal_test()


def buf_list_to_binary(buf_list):
    decimal_list = [buf_to_binary(buf) for i, buf in enumerate(buf_list) if i%2==0]
    return reverse_binary_to_decimal(decimal_list)

def buf_list_to_binary_test():
    assert(buf_list_to_binary('buffalo buffalo'.split())) == 0
    assert(buf_list_to_binary('Buffalo buffalo'.split())) == 1
    assert(buf_list_to_binary('buffalo buffalo Buffalo buffalo'.split())) == 2
    assert(buf_list_to_binary('Buffalo buffalo Buffalo buffalo'.split())) == 3

buf_list_to_binary_test()

def interpret(program):
    program = program.split('.')
    program = list(filter(lambda x: x not in ['','\n'], program))
    # check that each sentence parses. If not, just add more trailing buffalos
    for i, statement in enumerate(program):
        if not parses(statement):
            raise RuntimeError(f'Line {i+1}: "{statement}" does not parse')
    interpreted = []
    # The first two sentences are interpreted as a binary number
    # (except for the first two words, fixed as "Buffalo buffalo")
    # initialising the registers with a value.
    interpreted.append(f'buffalo {buf_list_to_binary(program[0].split()[2:])}')
    interpreted.append(f'Buffalo {buf_list_to_binary(program[1].split()[2:])}')
    interpreted.append('')
    for statement in program[2:]:
        statement = statement.split()
        # If the first variable token is Buffalo, instruction is JZ
        if statement[2] == 'Buffalo':
            instruction = 'JZ'
            # register to check for zero is second variable token
            register = statement[4]
            # rest of variable tokens binary encode line number to jump to
            Z = buf_list_to_binary(statement[6:])
            interpreted.append(' '.join([instruction, register, str(Z)]))
        else:
            # If first variable token not buffalo, second variable token
            # tells us whether instruction is INC or DEC
            if statement[4] == 'Buffalo':
                instruction = 'INC'
            elif statement[4] == 'buffalo':
                instruction = 'DEC'
            # register to act on is third variable token
            register = statement[6]
            interpreted.append(' '.join([instruction, register]))
    return '\n'.join(interpreted)

In [208]:
src = interpret(program)
print(src)

buffalo 2
Buffalo 0

JZ buffalo 4
DEC buffalo
INC Buffalo
JZ buffalo 0


In [209]:
from programmachine import Register, Instructions

In [210]:
instructions = Instructions(src)
print(f'Initial Register Values\n-----------------------')
registers = instructions.registers
for key, val in zip(registers.keys(),
                    [str(register.value) for register in registers.values()]):
                print(f'{key}: {val}')
print()
instructions_with_linenumbers = [f'{i+1}: {" ".join(instructions.instruction_list[i])}' for i in range(len((instructions.instruction_list)))]
instruction_print = '\n'.join(instructions_with_linenumbers)
print(f"""Instructions
------------
{instruction_print}
""")
print("Run program\n-----------")
instructions.interpret()

Initial Register Values
-----------------------
buffalo: 2
Buffalo: 0

Instructions
------------
1: JZ buffalo 4
2: DEC buffalo
3: INC Buffalo
4: JZ buffalo 0

Run program
-----------
line 1: register buffalo is not empty, doing nothing
line 2: decreasing register buffalo from 2 to 1
line 3: increasing register Buffalo from 0 to 1
line 4: register buffalo is not empty, doing nothing
line number out of range, halting

Final Register Values on Halt
-----------------------------
buffalo: 1
Buffalo: 1
